In [172]:
import filereader
import numpy as np
import gurobipy as gp
import scipy

In [173]:
groceries_dict = filereader.read_grocery_info()

In [174]:
recipes_dict = filereader.read_recipes()

In [175]:
nutrition_needs_dict = filereader.read_nutrition()

In [176]:
# generate the matrices that we care about
# we first need to be certain of order of stuff, so let's generate these lists of keys
groceries_list = [key for key in groceries_dict.keys()]
recipes_list = [key for key in recipes_dict.keys()]
nutrients_list = [key for key in nutrition_needs_dict.keys()]

# # generate the N matrix (but this will actually be a dictionary)
# N = {}
# for recipe, ingredients in recipes_dict.items():
#     N[recipe]

N = [
    [
        sum([groceries_dict[ingredient]["nutrients"][nutrient]/groceries_dict[ingredient]["Serving Size (g)"]*recipes_dict[recipe][ingredient] for ingredient in recipes_dict[recipe]])
    for nutrient in nutrients_list]
    for recipe in recipes_list
]

# c = np.array([groceries_dict[ingredient]["Price (CAD)"] for ingredient in groceries_list])

In [177]:
np.max(N)

6770.6

In [178]:
# need to transpose the recipes dict
recipes_dict_transpose = {}

for ik in groceries_dict.keys():
    recipes_dict_transpose[ik] = {}

for rk, rv in recipes_dict.items():
    for ik, iv in rv.items():
        recipes_dict_transpose[ik][rk] = iv

# first entry is a list, with an entry for each ingredient, containing a list of the indices of all the recipes
# the second entry is a list of lists of corresponding masses
recipes_transpose_csr_ish = [[],[]]
for ik, iv in recipes_dict_transpose.items():
    recipes_transpose_csr_ish[0].append([recipes_list.index(recipe) for recipe in iv.keys()])
    recipes_transpose_csr_ish[1].append([mass for mass in iv.values()])


In [179]:
N_dict = {}
for rk, rv in recipes_dict.items():
    nutrients_dict = {}
    for nutrient in nutrition_needs_dict.keys():
        nutrients_dict[nutrient] = 0
    for gk, gv in rv.items():
        for nk, nv in groceries_dict[gk]["nutrients"].items():
            nutrients_dict[nk] += nv*gv/groceries_dict[gk]["Serving Size (g)"]
    N_dict[rk] = nutrients_dict


In [180]:
# here, we define the cost, which is time-dependent
# for this reason, we need to be clear about the number of days we are considering

# the number of days we consider (one month)
# t_max = 30
t_max = 7

# the standard deviation for the random variation in price
price_sigma = 0.2

# costs for each item. Each is a list of 30 entries, one for each day
c_dict = {}
for gk, gv in groceries_dict.items():
    c_dict[gk] = gv["Price (CAD)"]*np.random.normal(loc=1, scale=price_sigma, size=t_max)

c_list = [gv["Price (CAD)"]*np.random.normal(loc=1, scale=price_sigma, size=t_max) for gk, gv in groceries_dict.items()]

# masses
m_dict = {}
for gk, gv in groceries_dict.items():
    m_dict[gk] = gv["Mass (g)"]

m_list = [gv["Mass (g)"] for gk, gv in groceries_dict.items()]

a_k = [nv for nk, nv in nutrition_needs_dict.items()]

In [181]:
model = gp.Model("New problem")

## seems that summation doesn't work unless we use the Gurobi addVars
# g_vars = {}
# u_vars = {}
# for gk in groceries_dict.keys():
#     u_vars[gk] = []
#     g_vars[gk] = model.addVars(t_max, vtype=gp.GRB.INTEGER, name=gk)
#     for t in range(t_max):
#         u_vars[gk].append(model.addVars(t+1, name=f"grams of {gk} used on day {t}"))


# r_vars = {}
# for rk in recipes_dict.keys():
#     r_vars[rk] = model.addVars(t_max, name=rk)

g_vars = model.addVars(len(groceries_list), t_max, vtype=gp.GRB.INTEGER, name="g[i][t]")
u_vars = model.addVars(len(groceries_list), t_max, t_max, name="u[i][tau][t]")
# r_vars = model.addVars(len(recipes_list), t_max, name="r[j][t]")
r_vars = model.addMVar((len(recipes_list), t_max), name="r[j][t]")

s = model.addVars(t_max, vtype=gp.GRB.BINARY, name="s")

model.setParam("LogToConsole", 1)  # Enable console logging
model.setParam("OutputFlag", 1)    # Enable all output

model.update()


Set parameter LogToConsole to value 1
Set parameter OutputFlag to value 1


In [182]:
(np.array(a_k)*np.expand_dims(np.arange(t_max), axis=1)).shape

(7, 16)

In [183]:
np.array(a_k)

array([3.0e+03, 5.6e+01, 2.3e+03, 4.7e+03, 1.3e+03, 4.1e+02, 1.1e+01,
       1.1e+01, 9.0e+02, 1.2e+00, 1.3e+00, 1.6e+01, 1.3e+00, 2.4e+00,
       7.5e+01, 1.5e+01])

In [184]:
np.max(np.array(N).transpose())

6770.6

In [185]:
np.max(m_list)

2000.0

In [186]:
for t in range(t_max):
    model.addMConstr(np.array(N).transpose(), r_vars[:,t], '>=', np.array(a_k))

model.addConstrs((g_vars[gk, t] <= 999999*s[t] for gk in range(len(groceries_list))
                  for t in range(t_max)), 
                  name="Buy only on shopping days")

for t in range(t_max):
    model.addConstrs(
        (
            # u_vars[i,tau,t] <= g_vars[i,tau]*m_list[i] - u_vars.sum(i,tau,'*') for i in range(len(groceries_list)) for tau in range(t)
            u_vars[i,tau,t] <= g_vars[i,tau]*m_list[i] - sum(u_vars[i,tau,gamma] for gamma in range(max(0,tau-1))) for i in range(len(groceries_list)) for tau in range(t+1)
        )
    )
    
# enforce that u matches what we're using
# for ik, R_i in enumerate(recipes_transpose_csr_ish):
for i, iv in enumerate(recipes_transpose_csr_ish[0]):
    masses_required = recipes_transpose_csr_ish[1][i]
    print(f"ingredient {i}/{len(recipes_transpose_csr_ish[0])}")
    for t in range(t_max):
        # model.addConstrs((
        model.addConstr((
            sum(r_vars[iv[ind], t]*masses_required[ind]  for ind in range(len(iv))) == sum(u_vars[i, tau, t] for tau in range(
                # (0 if t < int(groceries_dict[groceries_list[j]]["Shelf Life (Days)"]) 
                #     else t - int(groceries_dict[groceries_list[j]]["Shelf Life (Days)"])), 
                    t+1
                ))
        ))


model.update()

ingredient 0/49
ingredient 1/49
ingredient 2/49
ingredient 3/49
ingredient 4/49
ingredient 5/49
ingredient 6/49
ingredient 7/49
ingredient 8/49
ingredient 9/49
ingredient 10/49
ingredient 11/49
ingredient 12/49
ingredient 13/49
ingredient 14/49
ingredient 15/49
ingredient 16/49
ingredient 17/49
ingredient 18/49
ingredient 19/49
ingredient 20/49
ingredient 21/49
ingredient 22/49
ingredient 23/49
ingredient 24/49
ingredient 25/49
ingredient 26/49
ingredient 27/49
ingredient 28/49
ingredient 29/49
ingredient 30/49
ingredient 31/49
ingredient 32/49
ingredient 33/49
ingredient 34/49
ingredient 35/49
ingredient 36/49
ingredient 37/49
ingredient 38/49
ingredient 39/49
ingredient 40/49
ingredient 41/49
ingredient 42/49
ingredient 43/49
ingredient 44/49
ingredient 45/49
ingredient 46/49
ingredient 47/49
ingredient 48/49


In [187]:
recipes_dict_transpose

{'Beef': {'Zurich-Style Meat Saute': 600.0,
  'Hakka-Style Meatballs': 227.0,
  'Beef and Broccoli': 227.0},
 'Mushrooms': {'Zurich-Style Meat Saute': 250.0,
  'One-Pot Chicken Tetrazzini': 227.0},
 'Onion': {'Zurich-Style Meat Saute': 150.0,
  'Galinha Caipira': 150.0,
  'Grilled Mackerel with Miso Soup and Squash': 50.0,
  'Tajine Maadnous': 150.0,
  'Exotic Ginger Cumin Chicken': 75.0,
  'Hakka-Style Meatballs': 70.0,
  'One-Pot Chicken Tetrazzini': 70.0,
  'Smoked Salmon Pasta Primavera': 70.0,
  'Three Bean Salad': 70.0,
  'Fajitas': 150.0,
  'Smoked Salmon Quiche': 75.0},
 'Flour': {'Zurich-Style Meat Saute': 8.0,
  'Spiced Apple Pancakes': 250.0,
  'Grostoli': 800.0,
  'One-Pot Chicken Tetrazzini': 24.0,
  'Smoked Salmon Pasta Primavera': 24.0,
  'Spicy Kung Pao-Style Chicken': 30.0,
  'Smoked Salmon Quiche': 220.0},
 'Cream': {'Zurich-Style Meat Saute': 200.0, 'Smoked Salmon Quiche': 240.0},
 'Parsley': {'Zurich-Style Meat Saute': 5.0,
  'Tajine Maadnous': 50.0,
  'One-Pot Chic

In [201]:
g_vars

{(0, 0): <gurobi.Var g[i][t][0,0]>,
 (0, 1): <gurobi.Var g[i][t][0,1]>,
 (0, 2): <gurobi.Var g[i][t][0,2]>,
 (0, 3): <gurobi.Var g[i][t][0,3]>,
 (0, 4): <gurobi.Var g[i][t][0,4]>,
 (0, 5): <gurobi.Var g[i][t][0,5]>,
 (0, 6): <gurobi.Var g[i][t][0,6]>,
 (1, 0): <gurobi.Var g[i][t][1,0]>,
 (1, 1): <gurobi.Var g[i][t][1,1]>,
 (1, 2): <gurobi.Var g[i][t][1,2]>,
 (1, 3): <gurobi.Var g[i][t][1,3]>,
 (1, 4): <gurobi.Var g[i][t][1,4]>,
 (1, 5): <gurobi.Var g[i][t][1,5]>,
 (1, 6): <gurobi.Var g[i][t][1,6]>,
 (2, 0): <gurobi.Var g[i][t][2,0]>,
 (2, 1): <gurobi.Var g[i][t][2,1]>,
 (2, 2): <gurobi.Var g[i][t][2,2]>,
 (2, 3): <gurobi.Var g[i][t][2,3]>,
 (2, 4): <gurobi.Var g[i][t][2,4]>,
 (2, 5): <gurobi.Var g[i][t][2,5]>,
 (2, 6): <gurobi.Var g[i][t][2,6]>,
 (3, 0): <gurobi.Var g[i][t][3,0]>,
 (3, 1): <gurobi.Var g[i][t][3,1]>,
 (3, 2): <gurobi.Var g[i][t][3,2]>,
 (3, 3): <gurobi.Var g[i][t][3,3]>,
 (3, 4): <gurobi.Var g[i][t][3,4]>,
 (3, 5): <gurobi.Var g[i][t][3,5]>,
 (3, 6): <gurobi.Var g[i][t]

[array([6.32697202, 9.16006758, 5.86316648, 5.52832049, 7.64947743,
        7.29103384, 7.30882451]),
 array([3.29513311, 2.76224026, 1.75401078, 2.98984065, 2.68709035,
        2.24707282, 1.90056459]),
 array([0.54982914, 0.52032081, 0.60900535, 0.68193197, 0.58100119,
        0.35739405, 0.45088232]),
 array([4.46784599, 3.31758845, 4.88374436, 3.82403738, 3.94678954,
        2.82686921, 3.43131661]),
 array([3.09623187, 3.1557408 , 3.18685162, 2.42887922, 2.62333307,
        3.65763021, 2.60008072]),
 array([1.17031517, 1.31971228, 0.87529086, 0.87080095, 1.19813477,
        1.68367911, 0.96392876]),
 array([5.95858596, 7.57736326, 5.598887  , 5.57247538, 7.90327486,
        9.57740567, 9.58798642]),
 array([1.30935898, 1.35800591, 1.27626697, 1.86393428, 1.30347369,
        1.31752364, 1.46520688]),
 array([4.16508122, 4.48477385, 3.88667383, 6.50074774, 4.38854896,
        6.29910731, 4.68203826]),
 array([3.06137565, 2.50384277, 2.50496321, 2.67796641, 2.99287155,
        2.4112

In [188]:
# model.setObjective(g_vars.prod({k: v for k,v in enumerate(c_list)}) + 10*s.sum('*'))
model.setObjective(sum(g_vars))

model.update()

In [189]:
model.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 20.04.6 LTS")

CPU model: Intel(R) Core(TM) i7-10750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 2170 rows, 2877 columns and 9513 nonzeros
Model fingerprint: 0x8e8e61d3
Variable types: 2527 continuous, 350 integer (7 binary)
Coefficient statistics:
  Matrix range     [6e-02, 1e+06]
  Objective range  [1e+01, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+03]
Found heuristic solution: objective 70.0000000
Presolve removed 816 rows and 1493 columns
Presolve time: 0.06s
Presolved: 1354 rows, 1384 columns, 5589 nonzeros
Variable types: 1378 continuous, 6 integer (6 binary)

Root relaxation: objective 1.000000e+01, 12 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | I

In [190]:
np.max(model.getA())

6770.6

In [191]:
# Initialize variables to track the largest coefficient
max_coefficient = 0
max_row = None
max_col = None

# Iterate through all constraints
for i, constr in enumerate(model.getConstrs()):
    row = model.getRow(constr)  # Sparse representation of constraint row
    for j in range(row.size()):
        coeff = row.getCoeff(j)
        if abs(coeff) > abs(max_coefficient):  # Update if this coefficient is larger
            max_coefficient = coeff
            max_row = i
            max_col = row.getVar(j).index


In [192]:
max_coefficient

-999999.0

In [193]:
max(recipes_transpose_csr_ish[1])

[2500.0, 454.0, 907.0, 450.0, 680.0, 680.0, 454.0]

In [194]:
print(model.getVarByName("r[j][t][0,0]"))

<gurobi.Var r[j][t][0,0] (value 42.75534441805225)>


In [195]:
model.getVarByName("s")

In [196]:
# for var in model.getVars():
#     print(var)
for var in model.getVars():
        if var.x != 0:
            print(var.varName, "=", var.x)

g[i][t][0,0] = 999999.0
g[i][t][1,0] = 999999.0
g[i][t][2,0] = 999999.0
g[i][t][3,0] = 999999.0
g[i][t][4,0] = 999999.0
g[i][t][5,0] = 999999.0
g[i][t][6,0] = 999999.0
g[i][t][7,0] = 999999.0
g[i][t][8,0] = 999999.0
g[i][t][9,0] = 999999.0
g[i][t][10,0] = 999999.0
g[i][t][11,0] = 999999.0
g[i][t][12,0] = 999999.0
g[i][t][13,0] = 999999.0
g[i][t][14,0] = 999999.0
g[i][t][15,0] = 999999.0
g[i][t][16,0] = 999999.0
g[i][t][17,0] = 999999.0
g[i][t][18,0] = 999999.0
g[i][t][19,0] = 999999.0
g[i][t][20,0] = 999999.0
g[i][t][21,0] = 999999.0
g[i][t][22,0] = 999999.0
g[i][t][23,0] = 999999.0
g[i][t][24,0] = 999999.0
g[i][t][25,0] = 999999.0
g[i][t][26,0] = 999999.0
g[i][t][27,0] = 999999.0
g[i][t][28,0] = 999999.0
g[i][t][29,0] = 999999.0
g[i][t][30,0] = 999999.0
g[i][t][31,0] = 999999.0
g[i][t][32,0] = 999999.0
g[i][t][33,0] = 999999.0
g[i][t][34,0] = 999999.0
g[i][t][35,0] = 999999.0
g[i][t][36,0] = 999999.0
g[i][t][37,0] = 999999.0
g[i][t][38,0] = 999999.0
g[i][t][39,0] = 999999.0
g[i][t][40

In [197]:
print(model.getVarByName("g[i][t][0,0]"))

<gurobi.Var g[i][t][0,0] (value 999999.0)>


In [198]:
# for constr in model.getConstrs():
#         lhs_value = sum(var.X * coeff for var, coeff in zip(model.getVars(), model.getRow(constr)))
#         print(f"Constraint {constr.ConstrName}: LHS = {lhs_value:.2f}, RHS = {constr.RHS:.2f}")

In [199]:
# print(model.getRows())

# print(model.getConstrs())

for constr in model.getConstrs():
        print(model.getRow(constr))
        print(constr.RHS)

2035.92 r[j][t][0,0] + 6157.55 r[j][t][1,0] + 877.53 r[j][t][2,0] + 1215.88 r[j][t][3,0] + 1226.835 r[j][t][4,0] + 2205.895 r[j][t][5,0] + 4404.3 r[j][t][6,0] + 2465.93 r[j][t][7,0] + 1179.52 r[j][t][8,0] + 2404.32 r[j][t][9,0] + 1301.12 r[j][t][10,0] + 2137.04 r[j][t][11,0] + 1169.6100000000001 r[j][t][12,0] + 2744.5099999999998 r[j][t][13,0] + 1027.69 r[j][t][14,0] + 1588.365 r[j][t][15,0] + 4049.16 r[j][t][16,0] + 169.1 r[j][t][17,0]
3000.0
170.974 r[j][t][0,0] + 685.935 r[j][t][1,0] + 49.18600000000001 r[j][t][2,0] + 39.47 r[j][t][3,0] + 38.61000000000001 r[j][t][4,0] + 177.412 r[j][t][5,0] + 116.938 r[j][t][6,0] + 261.91200000000003 r[j][t][7,0] + 75.842 r[j][t][8,0] + 183.274 r[j][t][9,0] + 76.304 r[j][t][10,0] + 198.418 r[j][t][11,0] + 11.765 r[j][t][12,0] + 230.525 r[j][t][13,0] + 70.764 r[j][t][14,0] + 129.025 r[j][t][15,0] + 144.10399999999998 r[j][t][16,0] + 2.0900000000000003 r[j][t][17,0]
56.0
541.46 r[j][t][0,0] + 2059.15 r[j][t][1,0] + 2149.79 r[j][t][2,0] + 835.78 r[j][

-120.0 g[i][t][39,1] + u[i][tau][t][39,1,4]
0.0
-120.0 g[i][t][39,2] + u[i][tau][t][39,2,0] + u[i][tau][t][39,2,4]
0.0
-120.0 g[i][t][39,3] + u[i][tau][t][39,3,0] + u[i][tau][t][39,3,1] + u[i][tau][t][39,3,4]
0.0
-120.0 g[i][t][39,4] + u[i][tau][t][39,4,0] + u[i][tau][t][39,4,1] + u[i][tau][t][39,4,2] + u[i][tau][t][39,4,4]
0.0
-500.0 g[i][t][40,0] + u[i][tau][t][40,0,4]
0.0
-500.0 g[i][t][40,1] + u[i][tau][t][40,1,4]
0.0
-500.0 g[i][t][40,2] + u[i][tau][t][40,2,0] + u[i][tau][t][40,2,4]
0.0
-500.0 g[i][t][40,3] + u[i][tau][t][40,3,0] + u[i][tau][t][40,3,1] + u[i][tau][t][40,3,4]
0.0
-500.0 g[i][t][40,4] + u[i][tau][t][40,4,0] + u[i][tau][t][40,4,1] + u[i][tau][t][40,4,2] + u[i][tau][t][40,4,4]
0.0
-500.0 g[i][t][41,0] + u[i][tau][t][41,0,4]
0.0
-500.0 g[i][t][41,1] + u[i][tau][t][41,1,4]
0.0
-500.0 g[i][t][41,2] + u[i][tau][t][41,2,0] + u[i][tau][t][41,2,4]
0.0
-500.0 g[i][t][41,3] + u[i][tau][t][41,3,0] + u[i][tau][t][41,3,1] + u[i][tau][t][41,3,4]
0.0
-500.0 g[i][t][41,4] + u[i][tau

In [200]:
model.computeIIS()
model.write('iismodel.ilp')

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 20.04.6 LTS")

CPU model: Intel(R) Core(TM) i7-10750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

IIS computation: initial model status unknown, solving to determine model status
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 12 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%
IIS runtime: 0.02 seconds (0.00 work units)


GurobiError: Cannot compute IIS on a feasible model

In [ ]:
for tau in range(6):
    print(model.getVarByName(f"u[i][tau][t][48,{tau},6]"))

<gurobi.Var u[i][tau][t][48,0,6]>
<gurobi.Var u[i][tau][t][48,1,6]>
<gurobi.Var u[i][tau][t][48,2,6]>
<gurobi.Var u[i][tau][t][48,3,6]>
<gurobi.Var u[i][tau][t][48,4,6]>
<gurobi.Var u[i][tau][t][48,5,6]>
